In [255]:
#TO RE-RUN
%reset -f

In [256]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from datetime import datetime
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.4f}'.format
plt.style.use('classic')

%matplotlib inline



#### Required domain methods

In [261]:
import sys
sys.path.insert(1, "../src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter
import MLpipeline as MLpipeline
import readmision_methods as rm

#### Prepare experiments

In [277]:
typeEncounter = "last" # ['first','last']
typeHypothesis = "early_readmission_vs_none" # ['all_readmisssion_vs_none','early_readmission_vs_none']
typeDataFeatures = "extended_extra_diag_3" # ["reduced","extended','extended_extra','extended_extra_diag_1','extended_extra_diag_3']
    #Extended_Extra_diag_1 -> Extended extra columns with disease in diag_3
    #Extended_Extra_diag_1 -> Extended extra columns with disease in diag_1
    #Extended_Extra -> Extended extra of columns
    #Extended -> Extended columns
    #Reduced -> minimum set of columns 
typeDataExperiment = "all" #["all", "disease"] 
    #all -> No filter disease column
    #disease -> Filter/Remove disease column and keep only rows with diagnosis column == 1       

In [278]:
verbose = True
cv_thr = 0.3
cv_folds = 5

tr_thrs = [1.0] # [0.1,0.2,0.4,0.6,1.0]
ts_thr = 0.30

fs_methods = ["none","combine_fs","lasso_fs","rfe_rf_fs"] #["none","combine_fs","lasso_fs","rfe_rf_fs"]
cls_methods = ["logReg"] #["rf","svmRBF","logReg","knn","nn","gbt"]
lms = ["recall","f1"] #["f1_weighted","average_precision","roc_auc","recall"]
sm_types = ["none"] #["none","after"]
sm_method = "sm_smote"

In [279]:
#Load data
df_all = rm.load_data(typeEncounter, typeDataFeatures)
print "\nSHAPE:"
print df_all.shape

#Filter data by class
df_all = rm.filter_data_by_class(df_all, typeHypothesis)
print "\nSHAPE FILTERED:"
print df_all.shape

print "\nRows by class type:"
print df_all.iloc[:,-1].sort_values().unique(), np.sum(df_all["readmitted"] == 0), np.sum(df_all["readmitted"] == 1)

#Train & Test
X_train, X_test, y_train, y_test = MLpipeline.train_test_partition(df_all)
print "Train:", X_train.shape, "Test:",  X_test.shape

#Load hyperparams
hyperparams = np.load("../src/default_hyperparams.npy")
print "\nHyperparams:"
print hyperparams


SHAPE:
(67182, 69)

SHAPE FILTERED:
(45779, 69)

Rows by class type:
[0 1] 39785 5994
Train: (32045, 68) Test: (13734, 68)

Hyperparams:
[['fs' 'combine_fs' {'combine_fs__percentile': [5, 10, 20, 30, 40, 50]}]
 ['fs' 'rfe_rf_fs'
  {'rfe_rf_fs__n_features_to_select': [5, 10, 15, 20], 'rfe_rf_fs__step': [0.1]}]
 ['fs' 'lasso_fs' {'lasso_fs__estimator__C': [0.001, 0.01, 0.1, 1]}]
 ['cls' 'knn'
  {'knn__weights': ['uniform', 'distance'], 'knn__n_neighbors': [1, 3, 5, 7, 9, 11]}]
 ['cls' 'logReg'
  {'logReg__class_weight': [None, 'balanced'], 'logReg__C': [1e-05, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5, 10, 15, 30], 'logReg__penalty': ['l1', 'l2']}]
 ['cls' 'svmRBF'
  {'svmRBF__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 5], 'svmRBF__class_weight': [None, 'balanced'], 'svmRBF__C': [0.01, 0.1, 0.5, 1, 5, 10, 30, 50, 100]}]
 ['cls' 'rf'
  {'rf__criterion': ['entropy', 'gini'], 'rf__max_depth': [None, 4, 8, 12], 'rf__n_estimators': [200, 250, 300, 350, 400, 500]}]
 ['cls' 'nn'
  {'

In [ ]:
from IPython.display import display, HTML

res = []

experiments = rm.get_experiments(typeDataExperiment)
print experiments

for exp in experiments:
    
    print "\nEXPERIMENT:", exp
    print "******************\n"
    
    #Filter data by disease
    df_all_filtered = rm.filter_data_by_experiment(df_all, exp)
    
    #Get features by type
    catCols, reducedCols = rm.compute_type_features(df_all_filtered)
    
    print df_all_filtered.shape
    print df_all_filtered.columns
       
    #Create id of experiment
    name = exp + "_" + typeDataFeatures + "_" + typeEncounter + "_" + typeHypothesis    
    print "\n", name
    

    #Execute experiment
    res.append(MLpipeline.run(name, df_all_filtered, catCols, reducedCols, hyperparams, ts_thr, tr_thrs, 
                   fs_methods, sm_method, sm_types, 
                   cls_methods, lms, cv_folds, cv_thr, True, True))

['all']

EXPERIMENT: all
******************

(45779, 69)
Index([u'gender', u'age', u'race_AfricanAmerican', u'race_Caucasian',
       u'race_Other', u'HbA1c', u'Change', u'time_in_hospital', u'diabetesMed',
       u'diss_home', u'medSpec_cardio', u'medSpec_Family/GeneralPractice',
       u'medSpec_InternalMedicine', u'medSpec_surgery', u'adm_src_1',
       u'adm_src_2', u'adm_src_3', u'adm_src_4', u'adm_src_5', u'adm_src_6',
       u'adm_src_7', u'adm_src_8', u'adm_src_10', u'adm_src_11', u'adm_src_13',
       u'adm_src_14', u'adm_src_22', u'adm_src_25', u'adm_1', u'adm_2',
       u'adm_3', u'adm_4', u'adm_7', u'number_treatment',
       u'num_lab_procedures', u'num_procedures', u'num_medications',
       u'number_outpatient', u'number_emergency', u'number_inpatient',
       u'number_diagnoses', u'insulin', u'metformin', u'pioglitazone',
       u'glimepiride', u'glipizide', u'repaglinide', u'nateglinide',
       u'ComplexHbA1c', u'add_in_out', u'add_procs_meds', u'div_visits_time',
   

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 768 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1218 tasks      | elapsed:  8.7min
[Parallel(n_jobs=-1)]: Done 1680 out of 1680 | elapsed: 14.0min finished



TRAIN f1 (weighted): 0.659
TRAIN Precision [c=0,1]: [ 0.91484594  0.18851708]
TRAIN Recall [c=0,1]: [ 0.58637653  0.63775024]
TRAIN AUC: 0.612
TRAIN Sensibility: 0.637750238322
TRAIN Specificity:  0.586376530576

CV INNER metric: recall
CV INNER selected params ['balanced', 1e-05, 'l2', 50]
CV INNER score: 0.632883240667

CV OUTER f1-weighted score: 0.659  (+/-0.004)
CV OUTER prec score [c=0,1]: 0.914 (+/- 0.002), 0.187  (+/- 0.003)
CV OUTER rec  score [c=0,1]: 0.587 (+/- 0.006), 0.631  (+/- 0.012)
CV OUTER AUC score: 0.651  (+/-0.004)
CV OUTER Sensibility score: 0.631  (+/-0.012)
CV OUTER Specificity score: 0.587  (+/-0.006)
Selected params (bests from CV) ['balanced', 1e-05, 'l2', 50]

TEST f1 (weighted): 0.659
TEST Precision [c=0,1]: [ 0.91590134  0.18929974]
TEST Recall [c=0,1]: [ 0.5848693   0.64349277]
TEST AUC: 0.614
TEST Sensibility: 0.643492769744
TEST Specificity: 0.584869302949
Confussion matrix:
         | PRED
REAL-->  v 
[[6981 4955]
 [ 641 1157]]

Total time: 852.233366

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/il

In [201]:
if 'dfAux' not in locals():
    print "Not dfAux"
    dfAux = pd.DataFrame()

if 'df' in locals():
    print "Exist df"
    dfAux = df.copy()

df = pd.DataFrame(np.array(res[0]).reshape(len(res[0]),35), columns=
                          ["exp", "name",
                           "size_tr","fs","sm","cls","metric","params",
                           "tr_sens","tr_spec","tr_auc",
                           "tr_prec","tr_rec","tr_f1",
                           "cv_sens_mean","cv_sens_std","cv_spec_mean","cv_spec_std","cv_auc_mean","cv_auc_std",
                           "cv_prec_mean","cv_prec_std","cv_rec_mean","cv_rec_std",
                           "cv_f1_mean","cv_f1_std",
                           "test_sens","test_spec","test_auc",
                           "test_rec","test_prec","test_f1",
                           "cm_test",
                           "time","pipeline"])

df[["size_tr","fs","sm","metric","cls","params", 
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_std","cv_f1_mean","cv_sens_mean","cv_sens_std","cv_spec_mean","cv_spec_std","cv_auc_mean",
    "test_auc","test_sens","test_spec","test_f1"]]

Not dfAux


IndexError: list index out of range

In [30]:
dfAux =pd.DataFrame()
df = pd.concat((dfAux, df))
df[["size_tr","fs","sm","metric","cls","params", 
    "tr_auc","tr_sens","tr_spec",
    "cv_auc_mean","cv_f1_mean","cv_sens_mean","cv_spec_mean",
    "test_auc","test_f1","test_sens","test_spec"]].sort_values("cv_auc_mean", ascending=False)

,size_tr,fs,sm,metric,cls,params,tr_auc,tr_sens,tr_spec,cv_auc_mean,cv_f1_mean,cv_sens_mean,cv_spec_mean,test_auc,test_f1,test_sens,test_spec
5,0.0100,none,none,f1_weighted,logReg,"[None, 0.5, l1]",0.5577,0.1190,0.9964,0.6449,0.8142,0.0615,0.9880,0.5200,0.8178,0.0528,0.9872
3,0.0100,none,none,recall,logReg,"[balanced, 0.1, l1]",0.6882,0.8333,0.5432,0.6278,0.6403,0.5846,0.5735,0.5542,0.6062,0.5845,0.5238
4,0.0100,none,none,f1,logReg,"[balanced, 0.1, l1]",0.6882,0.8333,0.5432,0.6278,0.6403,0.5846,0.5735,0.5542,0.6062,0.5845,0.5238
2,0.0100,none,none,f1_weighted,rf,"[entropy, None, 200]",1.0000,1.0000,1.0000,0.5663,0.8018,0.0000,1.0000,0.5018,0.8093,0.0044,0.9992
8,0.0100,none,none,f1_weighted,knn,"[uniform, 9]",0.5238,0.0476,1.0000,0.5619,0.8018,0.0000,1.0000,0.5018,0.8091,0.0056,0.9981
1,0.0100,none,none,f1,rf,"[gini, 4, 200]",0.7850,0.5952,0.9748,0.5361,0.7868,0.0154,0.9566,0.5280,0.8115,0.1057,0.9504
0,0.0100,none,none,recall,rf,"[entropy, 4, 200]",0.7814,0.5952,0.9676,0.5312,0.7860,0.0154,0.9542,0.5291,0.8083,0.1190,0.9391
9,0.0100,none,none,recall,nb,[],0.5773,1.0000,0.1547,0.5188,0.3305,0.8154,0.2169,0.5001,0.2426,0.8576,0.1425
10,0.0100,none,none,f1,nb,[],0.5773,1.0000,0.1547,0.5188,0.3305,0.8154,0.2169,0.5001,0.2426,0.8576,0.1425
11,0.0100,none,none,f1_weighted,nb,[],0.5773,1.0000,0.1547,0.5188,0.3305,0.8154,0.2169,0.5001,0.2426,0.8576,0.1425
